## THIS IS FOR TESTING

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Dropout
import matplotlib.pyplot as plt

# Semilla para reproducibilidad
np.random.seed(42)

# ----------------------------------------------
# Paso 1: Cargar los Archivos CSV
# ----------------------------------------------

# Definir las rutas a los directorios donde están los archivos CSV
# Reemplaza estas rutas con las reales en tu sistema
directory_2024 = 'path/to/2024/data'  # Ejemplo: 'data/2024'
directory_2025 = 'path/to/2025/data'  # Ejemplo: 'data/2025'

# Lista de nombres de archivos CSV (22 carreras por temporada)
race_files_2024 = [f'race_{i}_2024.csv' for i in range(1, 23)]
race_files_2025 = [f'race_{i}_2025.csv' for i in range(1, 23)]

# Función para cargar los datos de una temporada
def load_season_data(directory, race_files):
    data = {}
    for race in race_files:
        file_path = os.path.join(directory, race)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            # Verificar que todas las columnas esperadas estén presentes
            expected_columns = ['Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 
                                'Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound', 
                                'TyreLife', 'Team', 'Position']
            if all(col in df.columns for col in expected_columns):
                data[race] = df
            else:
                print(f"Advertencia: El archivo {race} no tiene todas las columnas esperadas.")
        else:
            print(f"Archivo no encontrado: {file_path}")
    return data

# Cargar datos de 2024 y 2025
data_2024 = load_season_data(directory_2024, race_files_2024)
data_2025 = load_season_data(directory_2025, race_files_2025)

# ----------------------------------------------
# Paso 2: Procesar los Datos
# ----------------------------------------------

# Función para calcular el tiempo de pole position (mejor tiempo de la primera vuelta)
def calculate_pole_time(race_data):
    first_lap_times = race_data[race_data['LapNumber'] == 1]['LapTime']
    if not first_lap_times.empty:
        return first_lap_times.min()
    else:
        return np.nan

# Función para calcular el tiempo promedio por vuelta para cada piloto en una carrera
def calculate_avg_lap_time(race_data):
    # Filtrar vueltas atípicas (ejemplo: excluir tiempos mayores a 2 veces el tiempo de pole)
    pole_time = calculate_pole_time(race_data)
    if np.isnan(pole_time):
        return {}
    filtered_data = race_data[race_data['LapTime'] < 2 * pole_time]
    avg_lap_times = filtered_data.groupby('Driver')['LapTime'].mean().to_dict()
    return avg_lap_times

# Función para calcular el rendimiento normalizado p_{i,j} = tiempo_promedio / tiempo_pole
def calculate_performance(data):
    performance = {}
    for race, race_data in data.items():
        pole_time = calculate_pole_time(race_data)
        if np.isnan(pole_time):
            continue
        avg_lap_times = calculate_avg_lap_time(race_data)
        performance[race] = {driver: avg_time / pole_time for driver, avg_time in avg_lap_times.items()}
    return performance

# Calcular rendimiento para 2024 y 2025
performance_2024 = calculate_performance(data_2024)
performance_2025 = calculate_performance(data_2025)

# ----------------------------------------------
# Paso 3: Crear Secuencias para el Modelo LSTM
# ----------------------------------------------

# Función para crear secuencias de entrada
def create_sequences(performance, race_files, target_race_index):
    sequences = []
    additional_features = []
    targets = []
    drivers = list(performance[race_files[0]].keys())  # Pilotos de la primera carrera

    for driver in drivers:
        sequence = []
        for i in range(target_race_index):
            race = race_files[i]
            if race in performance and driver in performance[race]:
                sequence.append(performance[race][driver])
            else:
                sequence.append(np.nan)
        sequences.append(sequence)

        # Característica adicional: rendimiento en la misma carrera de 2024
        race_2024 = f'race_{target_race_index + 1}_2024.csv'
        if race_2024 in performance_2024 and driver in performance_2024[race_2024]:
            additional_features.append(performance_2024[race_2024][driver])
        else:
            additional_features.append(np.nan)

        # Objetivo: rendimiento en la carrera objetivo
        target_race = race_files[target_race_index]
        if target_race in performance and driver in performance[target_race]:
            targets.append(performance[target_race][driver])
        else:
            targets.append(np.nan)

    # Eliminar pilotos con datos faltantes
    valid_indices = [i for i in range(len(targets)) if not np.isnan(targets[i]) and not np.any(np.isnan(sequences[i])) and not np.isnan(additional_features[i])]
    sequences = [sequences[i] for i in valid_indices]
    additional_features = [additional_features[i] for i in valid_indices]
    targets = [targets[i] for i in valid_indices]

    return np.array(sequences), np.array(additional_features), np.array(targets), [drivers[i] for i in valid_indices]

# ----------------------------------------------
# Paso 4: Diseñar el Modelo LSTM
# ----------------------------------------------

# Parámetros ajustables de la red LSTM
lstm_units = 64          # Número de unidades en la capa LSTM
dense_units = 32         # Número de unidades en la capa densa adicional (opcional)
dropout_rate = 0.2       # Tasa de dropout para regularización
epochs = 50              # Número de épocas de entrenamiento
batch_size = 32          # Tamaño del batch
learning_rate = 0.001    # Tasa de aprendizaje

# Definir las entradas del modelo
sequence_input = Input(shape=(None, 1), name='sequence_input')      # Secuencia de rendimientos previos
additional_input = Input(shape=(1,), name='additional_input')       # Rendimiento en la misma carrera de 2024

# Capa LSTM para procesar la secuencia temporal
lstm_out = LSTM(lstm_units)(sequence_input)

# Agregar dropout para regularización
lstm_out = Dropout(dropout_rate)(lstm_out)

# Concatenar la salida de la LSTM con la característica adicional
concat = Concatenate()([lstm_out, additional_input])

# Capa densa adicional (opcional)
dense_out = Dense(dense_units, activation='relu')(concat)
dense_out = Dropout(dropout_rate)(dense_out)

# Capa de salida para predecir el rendimiento
output = Dense(1, activation='linear')(dense_out)

# Crear el modelo
model = Model(inputs=[sequence_input, additional_input], outputs=output)

# Compilar el modelo con optimizador Adam y pérdida de error cuadrático medio
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')

# Mostrar un resumen del modelo
model.summary()

# ----------------------------------------------
# Paso 5: Entrenamiento del Modelo
# ----------------------------------------------

# Crear datos de entrenamiento usando las carreras de 2024
def create_training_data(performance, race_files):
    X_seq_list = []
    X_add_list = []
    y_list = []

    for k in range(1, len(race_files)):
        sequences, additional_features, targets, _ = create_sequences(performance, race_files, k)
        if len(sequences) > 0:
            X_seq_list.append(sequences)
            X_add_list.append(additional_features)
            y_list.append(targets)

    if len(X_seq_list) == 0:
        return np.array([]), np.array([]), np.array([])

    X_seq_train = np.concatenate(X_seq_list, axis=0)
    X_add_train = np.concatenate(X_add_list, axis=0)
    y_train = np.concatenate(y_list, axis=0)

    return X_seq_train, X_add_train, y_train

# Generar datos de entrenamiento con datos de 2024
X_seq_train, X_add_train, y_train = create_training_data(performance_2024, race_files_2024)

# Ajustar las dimensiones para la entrada de la LSTM
X_seq_train = X_seq_train.reshape(X_seq_train.shape[0], X_seq_train.shape[1], 1)

# Entrenar el modelo
history = model.fit([X_seq_train, X_add_train], y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Visualizar la pérdida durante el entrenamiento
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.title('Pérdida del Modelo Durante el Entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida (MSE)')
plt.legend()
plt.show()

# ----------------------------------------------
# Paso 6: Predicción para una Carrera de 2025
# ----------------------------------------------

# Función para predecir el rendimiento en la carrera k de 2025
def predict_race_k(model, performance_2025, performance_2024, k):
    sequences, additional_features, _, drivers = create_sequences(performance_2025, race_files_2025, k-1)
    if len(sequences) == 0:
        print("No hay datos suficientes para hacer predicciones.")
        return []
    sequences = sequences.reshape(sequences.shape[0], sequences.shape[1], 1)
    predictions = model.predict([sequences, additional_features], verbose=0)
    return list(zip(drivers, predictions.flatten()))

# Ejemplo: predecir la carrera 5 de 2025
k = 5
predictions = predict_race_k(model, performance_2025, performance_2024, k)

if predictions:
    # Ordenar pilotos según el rendimiento predicho (menor p_{i,j} = mejor)
    sorted_drivers = sorted(predictions, key=lambda x: x[1])
    podium = [driver for driver, _ in sorted_drivers[:3]]

    # Mostrar el podio predicho
    print(f"\nPredicción del podio para la carrera {k} de 2025:")
    for i, driver in enumerate(podium, 1):
        print(f"Posición {i}: {driver}")
else:
    print("No se pudieron hacer predicciones para la carrera seleccionada.")

#### 1. Incluir Más Características en la Secuencia
#### Idea: Usa más variables como entradas para las secuencias.

#### Características:
#### Tiempos por sector (Sector1Time, Sector2Time, Sector3Time).

#### Compuesto de neumáticos (Compound, codificado como numérico: Soft=1, Medium=2, Hard=3).

#### Vida útil del neumático (TyreLife).



In [2]:
def calculate_features(race_data):
    pole_time = calculate_pole_time(race_data)
    features = race_data.groupby('Driver').agg({
        'LapTime': 'mean',
        'Sector1Time': 'mean',
        'Sector2Time': 'mean',
        'Sector3Time': 'mean',
        'TyreLife': 'mean',
        'Compound': lambda x: pd.Series(x).mode()[0]  # Compuesto más usado
    }).to_dict('index')
    for driver in features:
        features[driver]['LapTime'] /= pole_time
        features[driver]['Sector1Time'] /= pole_time
        features[driver]['Sector2Time'] /= pole_time
        features[driver]['Sector3Time'] /= pole_time
        # Codificar Compound
        compound_mapping = {'Soft': 1, 'Medium': 2, 'Hard': 3}
        features[driver]['Compound'] = compound_mapping.get(features[driver]['Compound'], 0)
    return features

#sequence_input = Input(shape=(None, 6), name='sequence_input')  # 6 características por paso


#### Idea: Calcula estadísticas por carrera como entradas adicionales.

#### Ejemplos:
#### Desviación estándar de LapTime (consistencia).

#### Número de stints (Stint único).



In [3]:
#additional_input = Input(shape=(3,), name='additional_input')  # Ejemplo: 3 características agregadas
#dense_out = Dense(64, activation='relu')(dense_out)
#dense_out = Dropout(dropout_rate)(dense_out)

==========================================================================

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

In [ ]:
# Semilla para reproducibilidad
np.random.seed(42)


# ----------------------------------------------
# Paso 1: Cargar los Datos
# ----------------------------------------------

In [ ]:
# Ruta al directorio con los datos (ajusta según tu sistema)
directory = 'path/to/2024/data'
race_files = [f'race_{i}_2024.csv' for i in range(1, 23)]  # 22 carreras

def load_data(directory, race_files):
    data = {}
    for race in race_files:
        file_path = os.path.join(directory, race)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            data[race] = df
        else:
            print(f"Archivo no encontrado: {file_path}")
    return data

data = load_data(directory, race_files)



# ----------------------------------------------
# Paso 2: Procesar los Datos
# ----------------------------------------------


In [ ]:

def calculate_pole_time(race_data):
    first_lap_times = race_data[race_data['LapNumber'] == 1]['LapTime']
    return first_lap_times.min() if not first_lap_times.empty else np.nan

def calculate_avg_lap_time(race_data):
    pole_time = calculate_pole_time(race_data)
    if np.isnan(pole_time):
        return {}
    filtered_data = race_data[race_data['LapTime'] < 2 * pole_time]
    return filtered_data.groupby('Driver')['LapTime'].mean().to_dict()

def calculate_performance(data):
    performance = {}
    for race, race_data in data.items():
        pole_time = calculate_pole_time(race_data)
        if np.isnan(pole_time):
            continue
        avg_lap_times = calculate_avg_lap_time(race_data)
        performance[race] = {driver: avg_time / pole_time for driver, avg_time in avg_lap_times.items()}
    return performance

performance = calculate_performance(data)


# ----------------------------------------------
# Paso 3: Dividir en Entrenamiento, Validación y Prueba
# ----------------------------------------------


In [ ]:
train_races = race_files[:15]  # Carreras 1-15
val_races = race_files[15:18]  # Carreras 16-18
test_races = race_files[18:]   # Carreras 19-22

def create_sequences(performance, race_files, target_race_index):
    sequences = []
    targets = []
    drivers = list(performance[race_files[0]].keys())
    
    for driver in drivers:
        sequence = []
        for i in range(target_race_index):
            race = race_files[i]
            sequence.append(performance.get(race, {}).get(driver, np.nan))
        sequences.append(sequence)
        
        target_race = race_files[target_race_index]
        targets.append(performance.get(target_race, {}).get(driver, np.nan))
    
    valid_indices = [i for i in range(len(targets)) if not np.isnan(targets[i]) and not np.any(np.isnan(sequences[i]))]
    sequences = [sequences[i] for i in valid_indices]
    targets = [targets[i] for i in valid_indices]
    drivers = [drivers[i] for i in valid_indices]
    
    return np.array(sequences), np.array(targets), drivers

# Datos de entrenamiento
X_train, y_train, drivers_train = create_sequences(performance, train_races, len(train_races) - 1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Datos de validación
X_val, y_val, drivers_val = create_sequences(performance, val_races, len(val_races) - 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

# Datos de prueba
X_test, y_test, drivers_test = create_sequences(performance, test_races, len(test_races) - 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


# ----------------------------------------------
# Paso 4: Diseñar y Entrenar el Modelo LSTM
# ----------------------------------------------


In [ ]:
sequence_input = Input(shape=(None, 1))
lstm_out = LSTM(64)(sequence_input)
lstm_out = Dropout(0.2)(lstm_out)
output = Dense(1, activation='linear')(lstm_out)

model = Model(inputs=sequence_input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Entrenar con validación
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                    epochs=50, batch_size=32, verbose=1)

# Graficar pérdida
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida Durante el Entrenamiento')
plt.xlabel('Época')
plt.ylabel('MSE')
plt.legend()
plt.show()



# ----------------------------------------------
# Paso 5: Predicción y Evaluación en Prueba
# ----------------------------------------------


In [ ]:
predictions = model.predict(X_test, verbose=0)

# Calcular métricas
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
spearman_corr, _ = spearmanr(y_test, predictions)
top3_true = np.argsort(y_test)[:3]
top3_pred = np.argsort(predictions.flatten())[:3]
accuracy_top3 = len(set(top3_true) & set(top3_pred)) / 3

print("\nMétricas de evaluación:")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"Correlación de Spearman: {spearman_corr:.4f}")
print(f"Precisión top-3: {accuracy_top3:.2%}")


In [ ]:
# Mostrar predicciones para la próxima carrera (ejemplo: carrera 19)
print(f"\nPredicciones para la carrera {test_races[0]}:")
sorted_predictions = sorted(zip(drivers_test, predictions.flatten()), key=lambda x: x[1])
for i, (driver, pred) in enumerate(sorted_predictions[:3], 1):
    print(f"Posición {i}: {driver}")